In [1]:
##import necessary libraries
import numpy as np
import pandas as pd
import time

In [2]:
#define a node class
class Node():
    def __init__(self, depth):
        self.leftChild = None
        self.rightChild = None
        self.giniSplitValue = None
        self.nodeGini = None
        self.cutoffValue = None
        self.cutoffLabel = None
        self.isLeaf = True
        self.depth = depth
        self.positives = 0
        self.negatives = 0
        
    #Takes in a series and returns a list of all the midpoints between each continuous data value
    def getListofMidpointsFromContinuousSeries(self, series):
        sortedSeries = series.sort_values()
        prevValue = sortedSeries.iloc[0]
        cutoffValues = []        
        # Get a list of the middle values between all unique data points
        for value in sortedSeries:
            # Just a check to see if the values are different 
            #so we don't calculate a ton of gini criterion options
            if value == prevValue:
                continue
            else:
                cutoffValues.append((value + prevValue)/2)
                prevValue = value
        return cutoffValues
    
    #Takes in a dataframe and label and calculates the gini coefficient for the continuous data represented by said label
    def calculateGiniForSingleContinuousLabel(self, df, label, classLabel):        
        cutoffValues = self.getListofMidpointsFromContinuousSeries(df[label])
        # Calulate gini coeff for each potential split
        weightedGiniCoeffs = {}
        for value in cutoffValues:
            leftChildPositiveCount = df[(df[label] <= value) & (df[classLabel] == 1)][label].count()
            rightChildPositiveCount = df[(df[label] > value) & (df[classLabel] == 1)][label].count()
            leftChildNegativeCount = df[(df[label] <= value) & (df[classLabel] == 0)][label].count()
            rightChildNegativeCount = df[(df[label] > value) & (df[classLabel] == 0)][label].count()
            leftTotal = leftChildPositiveCount + leftChildNegativeCount
            rightTotal = rightChildPositiveCount + rightChildNegativeCount
            giniLeft = 1 - (leftChildPositiveCount / leftTotal)**2 - (leftChildNegativeCount / leftTotal)**2
            giniRight = 1 - (rightChildPositiveCount / rightTotal)**2 - (rightChildNegativeCount / rightTotal)**2
            weightedGini = leftTotal/len(df) * giniLeft + rightTotal/len(df) * giniRight
            weightedGiniCoeffs[value] = weightedGini
            minGiniKey = min(weightedGiniCoeffs, key=weightedGiniCoeffs.get)
        if weightedGiniCoeffs == {}:
            return None
        else:
            return  (minGiniKey , weightedGiniCoeffs[minGiniKey])
    
    # Causes a node to calculate the gini coefficient for a best split for a given data frame
    # This populates most of the node's instance values
    def fitNode(self, df, classLabel):
        labels = list(df.columns)
        labels.remove(classLabel)
        giniCoeffs = {}
        for label in labels:
            #This is a dict of labels : tuples where the tuple is (split value, gini coeff)
            giniCoeffForLabel = self.calculateGiniForSingleContinuousLabel(df, label, classLabel)
            if giniCoeffForLabel is not None:
                giniCoeffs[label] = giniCoeffForLabel
        #Start with the first label in the list to find the min
        minGiniCoeffLabel = list(giniCoeffs.keys())[0]
        #The coefficient value is the second value in the tuple. The tuple is a value corresponding to said key
        minGiniCoeffValue = giniCoeffs[minGiniCoeffLabel][1]
        for label in giniCoeffs.keys():
            if giniCoeffs[label][1] <  minGiniCoeffValue:
                minGiniCoeffValue = giniCoeffs[label][1]
                minGiniCoeffLabel = label
        self.giniSplitValue = giniCoeffs[minGiniCoeffLabel][1]
        self.cutoffValue = giniCoeffs[minGiniCoeffLabel][0]
        self.cutoffLabel = minGiniCoeffLabel
    
    # Assigns the number of positives and negatives in a given node
    # Later we use this data to determine if an additional split is needed
    def assignCounts(self, df, classLabel):
        self.positives = len(df[df[classLabel] == 1])
        self.negatives = len(df[df[classLabel] == 0])
        self.nodeGini = 1 - (self.positives/(self.positives + self.negatives))**2 - (self.negatives/(self.positives + self.negatives))**2
    
    #Takes in a row of data (labels should match those that the tree was built with)
    #Returns a classification (1 for positive, zero for negative) for said row
    def classifyRow(self, row):
        if self.isLeaf == True:
            if self.positives >= self.negatives:
                return 1
            else:
                return 0
        else:
            if row[self.cutoffLabel] <= self.cutoffValue:
                return self.leftChild.classifyRow(row)
            else:
                return self.rightChild.classifyRow(row)
        

In [3]:
#Define a Decision tree class
class DTree():
    def __init__(self, maxDepth=500):
        if (maxDepth < 1):
            raise Exception("Max Tree depth must be >= 1")
        self.root = Node(depth=0)
        self.maxDepth = maxDepth
    
    # Fits each node in the tree recursively based on a given dataframe and corresponding class label
    def fit(self, df, classLabel):
        self.recursiveFit(df, classLabel, self.root)
    
    # Fits each node in the tree recursively based on a given dataframe and corresponding class label
    def recursiveFit(self, df, classLabel, node):
        node.assignCounts(df, classLabel)
        # max recursion depth reached
        if node.depth >= self.maxDepth:
            return
        #Node is already pure or not enough points to split on
        if node.positives == 0 or node.negatives == 0 or node.positives + node.negatives < 2:
            node.nodeGini = 0
            return
        
        # If we are not pure and haven't reached the max depth, determine splitting criteria
        node.fitNode(df, classLabel)
        
        #build data frames for next level of recursion
        dfLeft = df[df[node.cutoffLabel] <= node.cutoffValue]
        dfRight = df[df[node.cutoffLabel] > node.cutoffValue]
        
        #setup left child
        node.leftChild = Node(node.depth + 1)
        
        #setup right child
        node.rightChild = Node(node.depth + 1)
        
        #recurse
        node.isLeaf = False
        self.recursiveFit(dfLeft, classLabel, node.leftChild)
        self.recursiveFit(dfRight, classLabel, node.rightChild)
    
    # Give this function a data frame (labels need to be the same as the ones given during training)
    # and it will give you a list, equal in length to the data frame, of predictions
    def classify(self, df):
        predictions = []
        for i in range(len(df)):
            row = df.iloc[i]
            predictions.append(self.root.classifyRow(row))
        return predictions
            

In [4]:
#A lazy little function to calculate accuracy off a dataframe given a prediction label and a class label
def accuracy(df, classLabel, predictionLabel):
    return len(df[df[classLabel] == df[predictionLabel]]) / len(df)

In [5]:
#Get the diabetes data set to work with
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.isna().sum() #checking for any NA values

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [7]:
df.dtypes #checking dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

In [8]:
#Build a testTree with no max depth, time it just for fun
start = time.time()
testTree = DTree()
testTree.fit(df, "Outcome")
end = time.time()
elapsed_time = end - start
print("time:", elapsed_time, 'seconds')

time: 11.368341445922852 seconds


In [9]:
#Test the predictions on the training data
df['predictions'] = testTree.classify(df)

In [10]:
# We expect this should be 100% accurate
accuracy(df, "Outcome", "predictions")

1.0

In [11]:
#Checking data splits on a couple leaf nodes just to see if they are looking good
testNode = testTree.root
while testNode.isLeaf ==  False:
    testNode = testNode.leftChild
print(testNode.positives)
print(testNode.negatives)

testNode = testTree.root
while testNode.isLeaf ==  False:
    testNode = testNode.rightChild
print(testNode.positives)
print(testNode.negatives)

0
131
0
2


In [12]:
##calculate total data impurity to compare below
posCount = len(df[df["Outcome"] == 1])
negCount = len(df[df["Outcome"] == 0])
1 - (posCount/len(df))**2 - (negCount/len(df))**2

0.45437282986111116

In [13]:
print(testTree.root.cutoffLabel) #split label for root
print(testTree.root.cutoffValue) #split value for label
print(testTree.root.nodeGini) #Should be about .454, which is the data impurity
print(testTree.root.rightChild.cutoffLabel) #split label for right child
print(testTree.root.rightChild.cutoffValue) #split value for right child

Glucose
127.5
0.45437282986111116
BMI
29.95


In [14]:
#30% testing data, 70% training data
df = df.drop("predictions", axis=1)
test = df.sample(frac=0.30, random_state=1)
train = df[~df.index.isin(test.index)]
train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1


In [15]:
#Accuracy on no depth limit tree
realTree = DTree()
realTree.fit(train, "Outcome")
test['Predictions'] = realTree.classify(test)
accuracy(test, "Outcome", "Predictions")

0.691304347826087

In [16]:
print(realTree.root.cutoffLabel)
print(realTree.root.cutoffValue)
print(realTree.root.nodeGini)
print(realTree.root.rightChild.cutoffLabel)
print(realTree.root.rightChild.cutoffValue)
#Values seem to match Sklearn example from class, although the split may be different as different methods were used

Glucose
129.5
0.45007669877420153
BMI
27.85


In [17]:
#does limiting the depth help accuracy?
for i in range(1,9):
    realTree = DTree(i)
    realTree.fit(train, "Outcome")
    test['Predictions'] = realTree.classify(test)
    print(accuracy(test, "Outcome", "Predictions"))
#yes

0.7565217391304347
0.7608695652173914
0.7608695652173914
0.7913043478260869
0.7565217391304347
0.7391304347826086
0.7304347826086957
0.7217391304347827


Looks like a decision tree with a max depth of 4 gives us our best accuracy here at 79.13%. This means we have an error rate of 20.87%